# Explore the toraniko API

In [ ]:
import pandas as pd
import nasdaqdatalink as nasdaq
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns


## Fetch sample data

In [ ]:
price_data_df = nasdaq.get_table("SHARADAR/SEP")
price_data_df

In [ ]:
# Prep
price_data_df = price_data_df.sort_values(by=['ticker', 'date'])

price_data_df['asset_returns'] = price_data_df.groupby('ticker')['close'].pct_change()
price_data_df = price_data_df.dropna() # Lazy

price_data_df.rename(columns={'ticker': 'symbol'}, inplace=True)


In [ ]:
# Bodge into polars
price_data = pl.from_pandas(price_data_df)

# Run toraniko

In [ ]:
from toraniko.styles import factor_mom

#mom_df = factor_mom(price_data.select("symbol", "date", "asset_returns"), trailing_days=252, winsor_factor=0.01).collect()
# Using a small sample here (4 months) so reducing the trailing days. Need to consider the maths on this...
mom_df = factor_mom(price_data.select("symbol", "date", "asset_returns"), trailing_days=10, winsor_factor=0.01).collect()
mom_df

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(mom_df['mom_score'], kde=True, bins=50, color='skyblue', stat="density", linewidth=0)
plt.title('mom_score')
plt.xlabel('mom_score')
plt.ylabel('Density')
plt.show()